In [1]:
%cd ~/md4
import jax
import jax.numpy as jnp
import jax.random as jr

from importlib import reload
from md4.networks.transformer import Transformer
from md4.networks.transformer import ModelArgs
from md4.networks.hollow_transformer import HollowTransformer
from md4.networks.hollow_transformer import ModelArgs as HollowModelArgs

key_0 = jr.PRNGKey(0)

/home/yixiuz/md4


/home/yixiuz/md4_venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/yixiuz/md4_venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# Function to count parameters in a frozen dict
def count_parameters(frozen_dict):
    total_params = 0
    for value in frozen_dict.values():
        if isinstance(value, dict):  # If it's a nested dict
            total_params += count_parameters(value)
        else:  # It's an array
            total_params += value.size
    return total_params

In [3]:
feature_dim = 64
# This uses outside_embed with feature dim = 64
dummy_input = jnp.ones((64, 256, feature_dim))
cond = jnp.ones((64, feature_dim))

In [4]:
args = ModelArgs(dim=768,
          n_layers=12,
          n_heads=12,
          n_kv_heads=12,
          output_channels=27,
          hidden_dim=None,
          multiple_of=32,
          norm_eps=1e-05,
          dropout_rate=0.0,
          weight_tying=False,
          w_init_scale=1.0,
          depth_scaled_init=True,
          mlp_type='glu',
          cond_type='adaln_zero',
          embed_input=False,
          n_embed_classes=28,
          causal=False)
text8_transformer = Transformer(args)

In [5]:
# Initialize the transformer

params = text8_transformer.init(key_0, dummy_input, cond=cond, train=False)

In [6]:
count_parameters(params) # 85024512 = 85M

88699392

In [19]:
args = HollowModelArgs(dim=768,
          n_layers=12,
          n_heads=12,
          n_kv_heads=12,
          output_channels=27,
          # manually set hidden dim
          hidden_dim=2048, # 32 * 36
          multiple_of=32,
          norm_eps=1e-05,
          dropout_rate=0.0,
          weight_tying=False,
          w_init_scale=1.0,
          depth_scaled_init=True,
          mlp_type='glu',
          cond_type='adaln_zero',
          embed_input=False,
          n_embed_classes=28,
          causal=False,
          n_layers_per_mixed=12)

# ModelArgs(dim=768,
#           n_layers=12,
#           n_heads=12,
#           n_kv_heads=12,
#           output_channels=27,
#           hidden_dim=1152,
#           multiple_of=32,
#           norm_eps=1e-05,
#           dropout_rate=0.0,
#           weight_tying=False,
#           w_init_scale=1.0,
#           depth_scaled_init=True,
#           mlp_type='glu',
#           cond_type='adaln_zero',
#           embed_input=False,
#           n_embed_classes=28,
#           causal=False,
#           n_layers_per_mixed=6)
text8_hollow_transformer = HollowTransformer(args)

In [20]:
# This uses outside_embed with feature dim = 64
dummy_input = jnp.ones((64, 256, feature_dim))
params = text8_hollow_transformer.init(key_0, dummy_input, cond=None, train=False)

In [21]:
count_parameters(params) # 85024512 = 85M

101574144